### According to ChatGPT4, properties are python mechanisms called "descriptors" and they are to be defined at the class level NOT the instance level.
### So when we edit the class to create attrs, delete attrs, add setters/getters, we need to be smart. 

### Here is what we want to do:
# CLASS-LEVEL:
* Add `property`s for each getter-setter we want. This SHOULD NOT Clobber the existing instance attribute definitions, as long as the instance attributes are defined as self.attribute (or if using dataclasses, they should be type-hinted to imply that they are indeed instance attributes)

# INSTANCE-LEVEL:
* Delete existing attributes so when we access them, we're actually routed to the CLASS's `property`s with the same name
* Add new "storage" INSTANCE variables with the same name as the old ones but with an underscore like `_some_attr`

## OBSERVATIONS
* We have a problem. Decorators can't edit a class definition to include `property`s with the names of instance attributes until an instance is actually created, becuase when it is passed the class object, there is no way to inspect the `__intit__` method to see the attributes defined in its code. We have to wait until an instance is created and then use the `self` parameter to inspect instance attribute data.
* So, it seems like the only way to addresss this is to constantly overwrite the `property` attributes of the class with every new instance that is created. These `property` attributes will not give the class any new information, it will just be overwriting them each time...

# A hacky way to also do it, without re-defining the class every time an instance is created

* Inspect the source code of the `__init__` method, and use regex to find attribute names. Then we can add the properties to the class without looking at a created instance... 

# USING THE FIRST METHOD (re-defining the cls attributes each time an instance is created)


In [3]:
from dataclasses import dataclass

@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2

a1 = A()
print(vars(a1))

{'attr1': 'attr1', 'attr2': 2}


In [16]:
def a_decorator(cls):
    old_init = cls.__init__
    
    def new_init(self,*args,**kwargs):    # since cls is returned, this will just override cls.__init__ 
        print("----------In Init----------")
        property_descriptors_to_add_to_cls = dict()
        old_init(self,*args,**kwargs) # run old __init__ first
        # Now that we have some instance data, let's get it!
        old_instance_attrs_dict = {k:v for (k,v) in vars(self).items()}
        print(f"{old_instance_attrs_dict=}")
        for attrname, attrval in old_instance_attrs_dict.items():
            storage_name = f'_{attrname}'                            #
            setattr(self, storage_name, attrval)                     # The storage name is a true attribute, not a special property descriptor.
            # create the property so that we can add it to the cls
            property_descriptors_to_add_to_cls[attrname] = property(
                    lambda self: getattr(self, storage_name),         # fget a.k.a the getter
                    lambda self,val: setattr(self, storage_name,val), # fset a.k.a the setter
            )
            # delete the old attr becuase it has the same name as the property descriptor
            delattr(self,attrname)
        for attrname, property_defn in property_descriptors_to_add_to_cls.items():
            setattr(cls,attrname,property_defn)
        print(f"{vars(self).items()=}")
    cls.__init__ = new_init
    print("----------Exiting Decorator----------")
    return cls

@a_decorator
@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2
        
print("----------Done with class definition----------")

a1 = A()
print("----------created a1----------")
a2 = A()
print("----------created a2----------")
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # will be the same
a1.attr1 = "edited a1!"
a2.attr2 = "edited a2!"
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # seems like it works as expected!


----------Exiting Decorator----------
----------Done with class definition----------
----------In Init----------
old_instance_attrs_dict={'attr1': 'attr1', 'attr2': 2}
vars(self).items()=dict_items([('_attr1', 'attr1'), ('_attr2', 2)])
----------created a1----------
----------In Init----------
old_instance_attrs_dict={'_attr2': 2}
vars(self).items()=dict_items([('__attr2', 2)])
----------created a2----------
vars(a1)={'_attr1': 'attr1', '_attr2': 2}      vars(a2)={'__attr2': 2}
vars(a1)={'_attr1': 'attr1', '_attr2': 2, '__attr2': 'edited a1!'}      vars(a2)={'__attr2': 'edited a2!'}


### Lambda's are getting scope for storage_name from outside of their definition. So storage_name is whatever the last iteration of the loop was and things are getting fucked up

In [18]:
def a_decorator(cls):
    old_init = cls.__init__
    
    def create_property(instance, storage_name):
        return property(
                    lambda instance: getattr(instance, storage_name),         # fget a.k.a the getter
                    lambda instance,val: setattr(instance, storage_name,val), # fset a.k.a the setter
        )
    
    def new_init(self,*args,**kwargs):    # since cls is returned, this will just override cls.__init__ 
        print("----------In Init----------")
        property_descriptors_to_add_to_cls = dict()
        old_init(self,*args,**kwargs) # run old __init__ first
        # Now that we have some instance data, let's get it!
        old_instance_attrs_dict = {k:v for (k,v) in vars(self).items()}
        print(f"{old_instance_attrs_dict=}")
        for attrname, attrval in old_instance_attrs_dict.items():
            storage_name = f'_{attrname}'                            #
            setattr(self, storage_name, attrval)                     # The storage name is a true attribute, not a special property descriptor.
            # create the property so that we can add it to the cls
            property_descriptors_to_add_to_cls[attrname] = create_property(self,storage_name)
            # delete the old attr becuase it has the same name as the property descriptor
            delattr(self,attrname)
        for attrname, property_defn in property_descriptors_to_add_to_cls.items():
            setattr(cls,attrname,property_defn)
        print(f"{vars(self).items()=}")
    cls.__init__ = new_init
    print("----------Exiting Decorator----------")
    return cls

@a_decorator
@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2
        
print("----------Done with class definition----------")

a1 = A()
print("----------created a1----------")
a2 = A()
print("----------created a2----------")
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # will be the same
a1.attr1 = "edited a1!"
a2.attr2 = "edited a2!"
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # seems like it works as expected!


----------Exiting Decorator----------
----------Done with class definition----------
----------In Init----------
old_instance_attrs_dict={'attr1': 'attr1', 'attr2': 2}
vars(self).items()=dict_items([('_attr1', 'attr1'), ('_attr2', 2)])
----------created a1----------
----------In Init----------
old_instance_attrs_dict={'_attr1': 'attr1', '_attr2': 2}
vars(self).items()=dict_items([('__attr1', 'attr1'), ('__attr2', 2)])
----------created a2----------
vars(a1)={'_attr1': 'attr1', '_attr2': 2}      vars(a2)={'__attr1': 'attr1', '__attr2': 2}
vars(a1)={'_attr1': 'attr1', '_attr2': 2, '__attr1': 'edited a1!'}      vars(a2)={'__attr1': 'attr1', '__attr2': 'edited a2!'}


In [19]:
# Ok now underscores are being added an instances are still sharing data... No good

In [27]:
def a_decorator(cls):
    old_init = cls.__init__
    
    def create_property(storage_name):
        def g(instance): return getattr(instance,storage_name)
        def s(instance,value): return setattr(instance,storage_name,value)
        return property(g,s)
    
    def new_init(self,*args,**kwargs):    # since cls is returned, this will just override cls.__init__ 
        print("----------In Init----------")
        property_descriptors_to_add_to_cls = dict()
        old_init(self,*args,**kwargs) # run old __init__ first
        # Now that we have some instance data, let's get it!
        old_instance_attrs_dict = {k:v for (k,v) in vars(self).items()}
        print(f"{old_instance_attrs_dict=}")
        for attrname, attrval in old_instance_attrs_dict.items():
            storage_name = attrname if attrname.startswith('_') else f'_{attrname}'                            #
            setattr(self, storage_name, attrval)                     # The storage name is a true attribute, not a special property descriptor.
            # create the property so that we can add it to the cls
            property_descriptors_to_add_to_cls[attrname] = create_property(storage_name)
            # delete the old attr becuase it has the same name as the property descriptor
            delattr(self,attrname)
        for attrname, property_defn in property_descriptors_to_add_to_cls.items():
            setattr(cls,attrname,property_defn)
        print(f"{vars(self).items()=}")
    cls.__init__ = new_init
    print("----------Exiting Decorator----------")
    return cls

@a_decorator
@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2
        
print("----------Done with class definition----------")

a1 = A()
print("----------created a1----------")
a1.attr1 = "edited a1!"
print(f"{vars(a1)=}")

a2 = A()
print("----------created a2----------")
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # will be the same
a2.attr2 = "edited a2!"
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # seems like it works as expected!


----------Exiting Decorator----------
----------Done with class definition----------
----------In Init----------
old_instance_attrs_dict={'attr1': 'attr1', 'attr2': 2}
vars(self).items()=dict_items([('_attr1', 'attr1'), ('_attr2', 2)])
----------created a1----------
vars(a1)={'_attr1': 'edited a1!', '_attr2': 2}
----------In Init----------
old_instance_attrs_dict={'_attr1': 'attr1', '_attr2': 2}
vars(self).items()=dict_items([])
----------created a2----------
vars(a1)={'_attr1': 'edited a1!', '_attr2': 2}      vars(a2)={}


RecursionError: maximum recursion depth exceeded

In [49]:
def a_decorator(cls):
    
    old_init = cls.__init__
    cls.__PROPERTIES_ADDED = False
    
    def create_property(storage_name):
        def g(instance): return getattr(instance,storage_name)
        def s(instance,value): instance.__dict__[storage_name] = value
        return property(g,s)
    
    def new_init(self,*args,**kwargs):    # since cls is returned, this will just override cls.__init__ 
        print("----------In Init----------")
        print(f"{self.__PROPERTIES_ADDED=}")
        
        old_init(self,*args,**kwargs) # run old __init__ first
        print(f"{vars(self).items()=}")
        
        if not self.__class__.__PROPERTIES_ADDED:
            property_descriptors_to_add_to_cls = dict()

            # Inspect instance data
            og_instance_attrs_dict = {k:v for (k,v) in vars(self).items()}
            print(f"{og_instance_attrs_dict=}")
            for attrname, attrval in og_instance_attrs_dict.items():
                storage_name = attrname                           #
                setattr(self, storage_name, attrval)  # The storage name is a true attribute, not a special property descriptor.
                # create the property so that we can add it to the cls
                property_descriptors_to_add_to_cls[attrname] = create_property(storage_name)
                # delete the old attr becuase it has the same name as the property descriptor
                delattr(self,attrname)


            for attrname, property_defn in property_descriptors_to_add_to_cls.items():
                setattr(cls,attrname,property_defn)

            self.__class__.__PROPERTIES_ADDED = True
            
        if self.__class__.__PROPERTIES_ADDED:
            for attrname, attrval in vars(self).items():
                storage_name = attrname                           #
                setattr(self, storage_name, attrval)  # The storage name is a true attribute, not a special property descriptor.
                delattr(self,attrname)
            
        print(f"{vars(self).items()=}")
        
        
    ##################### END NEW_INIT #################### 
    cls.__init__ = new_init
    print("----------Exiting Decorator----------")
    return cls

@a_decorator
@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2
        
print("----------Done with class definition----------")

a1 = A()
print("----------created a1----------")
a1.attr1 = "edited a1!"
print(f"{vars(a1)=}")

a2 = A()
print("----------created a2----------")
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # will be the same
a2.attr2 = "edited a2!"
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # seems like it works as expected!
print(a2.attr1)

----------Exiting Decorator----------
----------Done with class definition----------
----------In Init----------
self.__PROPERTIES_ADDED=False
vars(self).items()=dict_items([('attr1', 'attr1'), ('attr2', 2)])
og_instance_attrs_dict={'attr1': 'attr1', 'attr2': 2}
vars(self).items()=dict_items([])
----------created a1----------
vars(a1)={'attr1': 'edited a1!'}
----------In Init----------
self.__PROPERTIES_ADDED=True
vars(self).items()=dict_items([('attr1', 'attr1'), ('attr2', 2)])


AttributeError: property of 'A' object has no deleter

# OPTION 2 (regex )

In [41]:
import re
import inspect

@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2


In [42]:
inspect.getsource(A.__init__)

OSError: could not get source code

# OKAY Back to option 1...

In [54]:
class B:
    pass

class A:
    attr1:str = "attr1"
    attr2:int = 2
    attr3: B = None

vars(A)


mappingproxy({'__module__': '__main__',
              '__annotations__': {'attr1': str,
               'attr2': int,
               'attr3': __main__.B},
              'attr1': 'attr1',
              'attr2': 2,
              'attr3': None,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [52]:
# Maybe go thru the __annotations__ and create getters and setters that way?

In [57]:
def a_decorator(cls):
    # class-level stuff
    for attrname in cls.__annotations__.keys():
        fget = lambda self, attrname=attrname: getattr(self, f"_{attrname}")
        fset = lambda self, value, attrname=attrname: setattr(self, f"_{attrname}", value)
        # set the property on the class
        setattr(cls, attrname, property(fget, fset))
    
    # make sure instances delete move all attributes defined in their __init__ to the new class properties
    # save a reference to the original __init__ method
    original_init = cls.__init__

    # define a new __init__ method
    def new_init(self, *args, **kwargs):
        # call the original __init__ method
        original_init(self, *args, **kwargs)

        # move attributes to their respective properties
        for attrname in cls.__annotations__.keys():
            value = getattr(self, attrname)
            delattr(self, attrname)
            setattr(self, attrname, value)

    # set the new __init__ method on the class
    cls.__init__ = new_init

    return cls

@a_decorator
@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2
        
print("----------Done with class definition----------")

a1 = A()
print("----------created a1----------")
a1.attr1 = "edited a1!"
print(f"{vars(a1)=}")

a2 = A()
print("----------created a2----------")
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # will be the same
a2.attr2 = "edited a2!"
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # seems like it works as expected!
print(a2.attr1)

----------Done with class definition----------


AttributeError: property of 'A' object has no deleter

In [90]:
def a_decorator(cls):
    # class-level stuff
    print("----------in decorator----------")
    for attrname in cls.__annotations__.keys():
        # capture the current attrname in the closure of the lambda functions
        fget = lambda self, attrname=attrname: getattr(self, f"_{attrname}")
        fset = lambda self, value, attrname=attrname: setattr(self, f"_{attrname}", value)
        fdel = lambda self, attrname=attrname: delattr(self, f"_{attrname}")
        # set the property on the class
        setattr(cls, attrname, property(fget, fset))

    # save a reference to the original __init__ method
    original_init = cls.__init__

    # define a new __init__ method
    def new_init(self, *args, **kwargs):
        print("----------in new init----------")
        # call the original __init__ method
        original_init(self, *args, **kwargs)

        # move attributes to their respective properties
        for attrname in cls.__annotations__.keys():
            print(f"looking at attrname {attrname}")
            value = getattr(self, attrname)
            # No need to delete the attribute, simply set the new attribute with a '_' prefix to value
            setattr(self, f'_{attrname}', value)

    # set the new __init__ method on the class
    cls.__init__ = new_init

    return cls

@a_decorator
@dataclass
class A:
    attr1: str = "attr1"
    attr2: int = 2

        
print("----------Done with class definition----------")

a1 = A()
print("----------created a1----------")
a1.attr1 = "edited a1!"
print(f"{vars(a1)=}")

a2 = A()
print("----------created a2----------")
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # will be the same
a2.attr2 = "edited a2!"
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # seems like it works as expected!
print(a2.attr1)

----------in decorator----------
----------Done with class definition----------
----------in new init----------
looking at attrname attr1
looking at attrname attr2
----------created a1----------
vars(a1)={'_attr1': 'edited a1!', '_attr2': 2}
----------in new init----------
looking at attrname attr1
looking at attrname attr2
----------created a2----------
vars(a1)={'_attr1': 'edited a1!', '_attr2': 2}      vars(a2)={'_attr1': 'attr1', '_attr2': 2}
vars(a1)={'_attr1': 'edited a1!', '_attr2': 2}      vars(a2)={'_attr1': 'attr1', '_attr2': 'edited a2!'}
attr1


In [91]:
print(vars(a1))
print(type(a1.attr1), type(A.attr1))

{'_attr1': 'edited a1!', '_attr2': 2}
<class 'str'> <class 'property'>


In [92]:
[(k,v) for k,v in A.__dict__.items() if not k.startswith('__')]

[('attr1', <property at 0x1262cb740>), ('attr2', <property at 0x1262cb470>)]

In [105]:
a3 = A()
print(a3)
vars(a3)

----------in new init----------
looking at attrname attr1
looking at attrname attr2
A(attr1='attr1', attr2=2)


{'_attr1': 'attr1', '_attr2': 2}

In [106]:
# delattr(a3,'_attr1') or 
del a3._attr1
vars(a3)

{'_attr2': 2}

In [97]:
a3.attr2

2

In [56]:
def a_decorator(cls):
    
    old_init = cls.__init__
    cls.__PROPERTIES_ADDED = False
    
    def create_property(storage_name):
        def g(instance): return getattr(instance,storage_name)
        def s(instance,value): instance.__dict__[storage_name] = value
        return property(g,s)
    
    def new_init(self,*args,**kwargs):    # since cls is returned, this will just override cls.__init__ 
        print("----------In Init----------")
        print(f"{self.__PROPERTIES_ADDED=}")
        
        old_init(self,*args,**kwargs) # run old __init__ first
        print(f"{vars(self).items()=}")
        
        if not self.__class__.__PROPERTIES_ADDED:
            property_descriptors_to_add_to_cls = dict()

            # Inspect instance data
            og_instance_attrs_dict = {k:v for (k,v) in vars(self).items()}
            print(f"{og_instance_attrs_dict=}")
            for attrname, attrval in og_instance_attrs_dict.items():
                storage_name = attrname                           #
                setattr(self, storage_name, attrval)  # The storage name is a true attribute, not a special property descriptor.
                # create the property so that we can add it to the cls
                property_descriptors_to_add_to_cls[attrname] = create_property(storage_name)
                # delete the old attr becuase it has the same name as the property descriptor
                delattr(self,attrname)


            for attrname, property_defn in property_descriptors_to_add_to_cls.items():
                setattr(cls,attrname,property_defn)

            self.__class__.__PROPERTIES_ADDED = True
            
        if self.__class__.__PROPERTIES_ADDED:
            for attrname, attrval in vars(self).items():
                storage_name = attrname                           #
                setattr(self, storage_name, attrval)  # The storage name is a true attribute, not a special property descriptor.
                delattr(self,attrname)
            
        print(f"{vars(self).items()=}")
        
        
    ##################### END NEW_INIT #################### 
    cls.__init__ = new_init
    print("----------Exiting Decorator----------")
    return cls

@a_decorator
@dataclass
class A:
    attr1:str = "attr1"
    attr2:int = 2
        
print("----------Done with class definition----------")

a1 = A()
print("----------created a1----------")
a1.attr1 = "edited a1!"
print(f"{vars(a1)=}")

a2 = A()
print("----------created a2----------")
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # will be the same
a2.attr2 = "edited a2!"
print(f"{vars(a1)=}", " "*4, f"{vars(a2)=}") # seems like it works as expected!
print(a2.attr1)

----------Exiting Decorator----------
----------Done with class definition----------
----------In Init----------
self.__PROPERTIES_ADDED=False
vars(self).items()=dict_items([('attr1', 'attr1'), ('attr2', 2)])
og_instance_attrs_dict={'attr1': 'attr1', 'attr2': 2}
vars(self).items()=dict_items([])
----------created a1----------
vars(a1)={'attr1': 'edited a1!'}
----------In Init----------
self.__PROPERTIES_ADDED=True
vars(self).items()=dict_items([('attr1', 'attr1'), ('attr2', 2)])


AttributeError: property of 'A' object has no deleter